## Imports, function definitions

Note: DeepMoD does not work for python3.9

In [1]:
# pip install pysindy

In [2]:
# pip install DeePyMoD

In [3]:
# pip install --upgrade tensorflow

In [4]:
# pip install tensorflow-io

In [5]:
results_path = ""

In [6]:
import matplotlib.pyplot as plt

# General imports
import numpy as np
import torch
import pandas as pd

# DeePyMoD imports
from deepymod import DeepMoD
from deepymod.data import Dataset, get_train_test_loader
from deepymod.data.samples import Subsample_random
from deepymod.data.burgers import burgers_delta
from deepymod.model.constraint import LeastSquares
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.sparse_estimators import Threshold
from deepymod.training import train
from deepymod.training.sparsity_scheduler import Periodic, TrainTest, TrainTestPeriodic

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(device)

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import pickle
from sklearn.metrics import mean_squared_error
def load_data(X,y):
    return torch.from_numpy(X).float(),torch.from_numpy(y).float()

def uxt_2D_to1D(u,x,t):
    '''
    Function for reshaping u,x and t
    '''
    t_reg=[]
    x_reg=[]
    for i in range(len(t)):
        for j in range(len(x)):
            t_reg.append(t[i])
            x_reg.append(x[j])
    u_reg = u.reshape((u.size, 1))

    return u_reg,x_reg,t_reg

descr = ['',
 'u_{x}',
 'u_{xx}',
 'u_{xxx}',
 'u',
 'uu_{x}',
 'uu_{xx}',
 'uu_{xxx}',
 'u^2',
 'u^2u_{x}',
 'u^2u_{xx}',
 'u^2u_{xxx}']

def return_pde(w, rhs_description=descr, ut = 'u_t'):
    pde = ut + ' = '
    first = True
    for i in range(len(w)):
        if w[i] != 0:
            if not first:
                pde = pde + ' + '
            pde = pde + "(%05f)" % (w[i].real) + rhs_description[i] 
            first = False
    return pde


cuda


In [7]:
def error_metric(true_coeff,pred_coeff):
    wrmse = mean_squared_error(true_coeff,pred_coeff,squared=False)
    return wrmse

## Testing 1.1

In [8]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(1,3):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  20000
Dataset is using device:  cuda


2023-04-28 10:52:43.284522: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 10:52:43.796440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 19750  MSE: 6.29e+03  Reg: 1.63e-11  L1: 2.71e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.407732) + (-914274.125000)u_{x} + (1045492.312500)u_{xx} + (218240.500000)u_{xxx} + (0.001999)u + (-3421.692627)uu_{x} + (-3355.112549)uu_{xx} + (-62.208099)uu_{xxx} + (-0.000002)u^2 + (21.269197)u^2u_{x} + (-0.321490)u^2u_{xx} + (-2.174442)u^2u_{xxx}  rmse =  405853.3111874253
Dataset is using device:  cuda
 10250  MSE: 1.05e+04  Reg: 1.31e-10  L1: 1.28e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000012) + (-0.843485)u_{x} + (0.464392)u_{xxx}  rmse =  0.28131040541315533
Dataset is using device:  cuda
 19750  MSE: 8.99e+04  Reg: 8.46e-14  L1: 7.15e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (1260574.125000)u_{x} + (-0.012431)u_{xx} + (-104501.625000)u_{xxx} + (11669.486328)uu_{x} + (-248.435913)uu_{xxx} + (-1402.244629)u^2u_{x} + (95.364929)u^2u_{xxx}  rmse =  365160.45882406464
Dataset is using device:  cuda
 1

KeyboardInterrupt: 

In [8]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(2,3):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  15000
Dataset is using device:  cuda


2023-04-30 12:08:01.094579: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 12:08:01.787292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  1250  MSE: 1.07e+05  Reg: 1.07e-14  L1: 9.86e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.122730)u_{xx} + (0.007611)u_{xxx}  rmse =  0.07876088549496463
Dataset is using device:  cuda
 19750  MSE: 1.08e+05  Reg: 5.16e-03  L1: 4.07e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000034) + (48567.730469)u_{xx} + (-1295.577515)uu_{x} + (-12768.605469)uu_{xx} + (166.966583)u^2u_{x} + (839.920410)u^2u_{xx}  rmse =  14503.617432204288
Dataset is using device:  cuda
  2250  MSE: 1.06e+05  Reg: 6.44e-04  L1: 1.78e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-14375.561523)u_{xx} + (5658.698242)u_{xxx} + (-8554.548828)uu_{x} + (12962.596680)uu_{xx} + (5551.987793)uu_{xxx} + (996.413391)u^2u_{x} + (-1334.440063)u^2u_{xx} + (-717.736938)u^2u_{xxx}  rmse =  6544.7463067039425
Dataset is using device:  cuda
  1500  MSE: 1.06e+05  Reg: 3.79e-10  L1: 4.66e+04 Algorithm converged. Writing model to disk.
Predicted PDE:  u_

## Testing 1.2

In [9]:
# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(3,5):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  10000
Dataset is using device:  cuda
  2500  MSE: 1.06e+05  Reg: 1.36e-04  L1: 4.70e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000000) + (455550.312500)u_{xx} + (213829.875000)u_{xxx} + (21020.304688)uu_{x} + (-87250.460938)uu_{xx} + (-53629.847656)uu_{xxx} + (-2123.573975)u^2u_{x} + (4123.315430)u^2u_{xx} + (3231.496582)u^2u_{xxx}  rmse =  148383.4493706548
Dataset is using device:  cuda
  1500  MSE: 1.05e+05  Reg: 4.35e-13  L1: 9.24e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.240804)u_{xx}  rmse =  0.11281540208934591
Dataset is using device:  cuda
 19750  MSE: 1.06e+05  Reg: 3.61e-02  L1: 9.79e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000009) + (-206925.984375)u_{xx} + (53390.355469)u_{xxx} + (-7484.995117)uu_{x} + (44366.773438)uu_{xx} + (-13262.300781)uu_{xxx} + (885.583862)u^2u_{x} + (-2362.092773)u^2u_{xx} + (824.021179)u^2u_{xxx}  rmse =  63164.0207816011
Dataset is using device:  

## Testing 1.3

In [10]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(5,7):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  5000
Dataset is using device:  cuda
  1500  MSE: 1.07e+05  Reg: 7.35e-11  L1: 7.99e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000005) + (0.424230)u_{xxx}  rmse =  0.1298946381571819
Dataset is using device:  cuda
  2500  MSE: 1.07e+05  Reg: 6.22e-04  L1: 3.49e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000013) + (704559.437500)u_{xx} + (131919.500000)u_{xxx} + (116.058548)uu_{x} + (-171596.890625)uu_{xx} + (-32518.173828)uu_{xxx} + (-14.130068)u^2u_{x} + (10445.720703)u^2u_{xx} + (2001.533447)u^2u_{xxx}  rmse =  212998.94394699775
Dataset is using device:  cuda
  2000  MSE: 1.06e+05  Reg: 5.85e-14  L1: 9.76e-01 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (-0.151115)u_{xx} + (0.035605)u_{xxx}  rmse =  0.08752994210636782
Dataset is using device:  cuda
  2250  MSE: 1.07e+05  Reg: 1.41e-03  L1: 2.56e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000014) + (108437.500000)u_{xx}

## Testing 1.4

In [11]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(7,9):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  2000
Dataset is using device:  cuda
  1750  MSE: 1.05e+05  Reg: 1.33e-09  L1: 1.08e+00 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000017) + (-0.695317)u_{x} + (-0.666692)u_{xx}  rmse =  0.30963041324617163
Dataset is using device:  cuda
  1250  MSE: 1.07e+05  Reg: 5.01e-06  L1: 2.67e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (154745.750000)u_{xx} + (33897.382812)u_{xxx} + (-6083.073242)uu_{x} + (-1241.845947)uu_{xx} + (1714.587646)uu_{xxx} + (640.437073)u^2u_{x} + (-1570.521240)u^2u_{xx} + (-553.290588)u^2u_{xxx}  rmse =  45771.0806443386
Dataset is using device:  cuda
  2500  MSE: 1.06e+05  Reg: 5.87e-05  L1: 9.92e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (486374.500000)u_{x} + (443686.562500)u_{xx} + (192378.656250)u_{xxx} + (116270.375000)uu_{x} + (-34167.484375)uu_{xx} + (-24647.066406)uu_{xxx} + (-22502.554688)u^2u_{x} + (-2752.729248)u^2u_{xx} + (38.470257)u^2u_{xxx}  rmse =  201294.7617316568

## Testing 1.5

In [8]:


# Heat equation ut = 0.15uxx

snrs = ["No noise",20000,15000,10000,7500,5000,3000,2000,1000,800,500,300,200,100,90,80,70,60,50,40,30,20,10,8,6,5,4,3,2,1,0.9,0.8,0.75,0.7,0.6,0.5,0.4,0.3,0.25,0.2,0.1]

true_coeff = [0]*12
true_coeff[2]=0.15

num_realizations=30

avg_rmses = []

# Loading u,x,t
file_to_read = open("1_"+str(0)+".pkl", "rb")
loaded_dictionary = pickle.load(file_to_read)
uc = loaded_dictionary["u"]
x = loaded_dictionary["x"]
t = loaded_dictionary["t"]
dx = x[2]-x[1]
dt = t[2]-t[1]

for snrid in range(9,11):
    
    print("="*100)
    print("SNR = ",snrs[snrid])
    
    rmses=[]
        
    # Adding noise
    for _ in range(num_realizations):
        
        # To ensure that random values are changed each time
        np.random.seed()
        
        u_noisy = uc + np.random.normal(0,np.sqrt(np.var(uc)/float(snrs[snrid])), uc.shape)
        
        # reshaping
        u_reg,x_reg,t_reg = uxt_2D_to1D(u_noisy,x,t)
        X = np.vstack([t_reg,x_reg]).T
        y = u_reg

        # Deepmod
        preprocess_kwargs = {"noise_level": 0.0}
        load_kwargs = {"X": X, "y": y}
        dataset = Dataset(
        load_data,
        load_kwargs=load_kwargs,
        preprocess_kwargs=preprocess_kwargs,
        subsampler=Subsample_random,
        subsampler_kwargs={"number_of_samples": 5000},
        device=device,
        )
        train_dataloader, test_dataloader = get_train_test_loader(
        dataset, train_test_split=0.8
        )
        network = NN(2, [50, 50, 50,50], 1)
        library = Library1D(poly_order=2, diff_order=3) 
        estimator = Threshold(0.1) 
        sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=200, delta=1e-5) 
        constraint = LeastSquares() 
        model = DeepMoD(network, library, estimator, constraint).to(device)
        optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) 
        train(
          model,
          train_dataloader,
          test_dataloader,
          optimizer,
          sparsity_scheduler,
          exp_ID="Test",
          write_iterations=250,
          max_iterations=20000,
          delta=1e-4,
          patience=200,
        )

        w = list(model.constraint_coeffs()[0])
        w = [w[i][0].item() for i in range(len(w))]
        
        rmse = error_metric(true_coeff,w)
        print("Predicted PDE: ", return_pde(w, descr), " rmse = ",rmse)
        rmses.append(rmse)
    print("\n\n\n SNR = ", snrs[snrid], ": Average rmse =  ", np.mean(rmses))
    avg_rmses.append(np.mean(rmses))
    

SNR =  800
Dataset is using device:  cuda


2023-05-05 16:10:17.600990: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 16:10:18.193083: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  2250  MSE: 1.06e+05  Reg: 9.11e-05  L1: 4.30e+06 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000000) + (-61592.234375)u_{xx} + (-50943.273438)u_{xxx} + (-4365.906250)uu_{x} + (4019.546875)uu_{xx} + (14770.190430)uu_{xxx} + (542.108398)u^2u_{x} + (441.100586)u^2u_{xx} + (-1047.650024)u^2u_{xxx}  rmse =  23529.770793499043
Dataset is using device:  cuda
 19750  MSE: 1.03e+05  Reg: 5.22e-05  L1: 1.99e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (0.000016) + (-81694.898438)u_{xx} + (916.674805)uu_{x} + (11324.931641)uu_{xx} + (-60.581100)u^2u_{x} + (-391.386902)u^2u_{xx}  rmse =  23810.59316485089
Dataset is using device:  cuda
  2250  MSE: 1.06e+05  Reg: 3.77e-02  L1: 8.95e+05 Algorithm converged. Writing model to disk.
Predicted PDE:  u_t = (214813.281250)u_{x} + (-60920.781250)u_{xx} + (-55984.402344)u_{xxx} + (-63936.078125)uu_{x} + (-1739.910156)uu_{xx} + (9460.617188)uu_{xxx} + (4612.772949)u^2u_{x} + (1161.636719)u^2u_{xx} + (-316.